In [15]:
#  import modules
import pandas as pd
from scapy.all import *
import plotly.express as px
import networkx as nx

In [16]:
PCAP_FILE = "./Datasets/Botnet Data/mirai_botnet.pcap"

In [17]:
pc = rdpcap(PCAP_FILE)

In [18]:
# number of packets
len(pc)

34612

In [27]:
pc[1].layers()

[scapy.layers.l2.Ether, scapy.layers.inet.IP, scapy.layers.inet.TCP]

In [31]:
pc[1].getlayer(2).payload

In [34]:
pc[1].getlayer(2).fields

{'sport': 23,
 'dport': 50385,
 'seq': 2788395614,
 'ack': 2503511369,
 'dataofs': 6,
 'reserved': 0,
 'flags': <Flag 18 (SA)>,
 'window': 29200,
 'chksum': 52910,
 'urgptr': 0,
 'options': [('MSS', 1460)]}

In [56]:
def extract_fields(packet):
    res = {}
    layers = packet.layers()
    for i in range(len(layers)):
        layer = packet.getlayer(i)
        res.update(layer.fields)
    return res

In [57]:
packets = list(map(extract_fields, pc))

In [58]:
df = pd.DataFrame(packets)

In [59]:
df.shape

(34612, 23)

In [65]:
df.head()

,dst,src,type,options,version,ihl,tos,len,id,flags,...,chksum,sport,dport,seq,ack,dataofs,reserved,window,urgptr,load
0,192.168.0.2,79.37.47.139,2048,[],4,5,0,40,65077,S,...,46170,50385,23,2503511368,0,5,0,54467,0,NaN
1,79.37.47.139,192.168.0.2,2048,"[(MSS, 1460)]",4,5,0,44,0,SA,...,52910,23,50385,2788395614,2503511369,6,0,29200,0,NaN
2,192.168.0.2,79.37.47.139,2048,[],4,5,0,40,0,R,...,35099,50385,23,2503511369,0,5,0,0,0,NaN
3,192.168.0.2,79.37.47.139,2048,"[(MSS, 1452), (SAckOK, b''), (Timestamp, (1078...",4,5,0,60,25760,S,...,12478,45122,23,2064578796,0,10,0,5808,0,NaN
4,79.37.47.139,192.168.0.2,2048,"[(MSS, 1460), (SAckOK, b''), (Timestamp, (2920...",4,5,0,60,0,SA,...,51736,23,45122,1075730922,2064578797,10,0,28960,0,NaN


In [68]:
df.columns

Index(['dst', 'src', 'type', 'options', 'version', 'ihl', 'tos', 'len', 'id',
       'flags', 'frag', 'ttl', 'proto', 'chksum', 'sport', 'dport', 'seq',
       'ack', 'dataofs', 'reserved', 'window', 'urgptr', 'load'],
      dtype='object')

## Tabular Analysis

In [69]:
df.groupby("src").sum().sort_values(by="len", ascending=False)

,type,version,ihl,tos,len,id,frag,ttl,proto,chksum,sport,dport,seq,ack,dataofs,reserved,window,urgptr
src,,,,,,,,,,,,,,,,,,
192.168.0.2,59615232,116436,145545,2080,1546108,971206167,0,1862976,174654,959017296,597474648,31246774,58343559115092,8878253357180,155820,0,845735159,0
209.255.80.252,677888,1324,1655,0,19466,10203802,0,80433,1986,12075033,7613,13931318,656220697289,895909076689,1665,0,1352564,0
208.101.209.216,536576,1048,1310,0,15929,7255953,0,13362,1572,7924796,6026,10811016,541257500730,720971508535,2114,0,790656,0
80.252.139.73,638976,1248,1560,0,14484,15087694,0,34944,1872,10964516,7176,15445552,420515549709,770965302928,1560,0,1277952,0
84.196.158.177,524288,1024,1280,0,14064,9606922,0,12544,1536,9908245,11108531,5888,768933402866,584367169416,2068,0,778620,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
175.92.42.112,2048,4,5,0,40,13897,0,51,6,63810,23,16946,0,2942052977,5,0,0,0
175.94.186.171,2048,4,5,0,40,56963,0,51,6,55495,23,16946,0,2942220972,5,0,0,0
176.142.185.214,2048,4,5,0,40,0,0,49,6,55313,23,16946,0,2962143703,5,0,0,0


In [78]:
fig = px.bar(df, x="dst", y="len",color=df.dport.astype(str), title="Destination Traffic by Port")
fig.show()

In [70]:
df.groupby("dst").sum().sort_values(by="len", ascending=False)

,type,version,ihl,tos,len,id,frag,ttl,proto,chksum,sport,dport,seq,ack,dataofs,reserved,window,urgptr
dst,,,,,,,,,,,,,,,,,,
93.190.142.201,571392,1116,1395,0,343297,17472609,0,17856,1674,9122330,476467,9082746,91042364100,1066730906490,2236,0,618071,0
192.168.0.2,11270144,22012,27515,124328,299981,146890676,0,587465,33018,187052107,27087063,199671976,9746071250310,11584338885600,36524,0,29004358,1
84.196.158.177,538624,1052,1315,0,15449,9022617,0,16832,1578,10264157,6049,11432985,630472380237,789189500687,2122,0,774920,0
208.101.209.216,559104,1092,1365,0,14835,8952401,0,17472,1638,8427029,11241530,6279,751955533764,539292398240,2198,0,800335,0
88.249.164.167,499712,976,1220,0,13338,10181545,0,15616,1464,8299218,11936470,5612,579871764572,303039436482,1967,0,720035,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
165.172.19.89,2048,4,5,0,40,51776,0,64,6,48601,16946,23,2779517785,0,5,0,32010,0
165.171.230.174,2048,4,5,0,40,26173,0,64,6,5936,16946,23,2779506350,0,5,0,32010,0
165.171.105.58,2048,4,5,0,40,22170,0,64,6,4633,16946,23,2779474234,0,5,0,32010,0


In [67]:
df.groupby("sport").sum().sort_values(by="len", ascending=False)

,type,version,ihl,tos,len,id,frag,ttl,proto,chksum,dport,seq,ack,dataofs,reserved,window,urgptr
sport,,,,,,,,,,,,,,,,,
16946,49969152,97596,121995,2080,975960,799429076,0,1561536,146394,800459429,561177,48364178260578,0,121995,0,778483200,0
23,11474944,22412,28015,124328,635225,154917592,0,602941,33618,190558113,230251543,9185364358439,12619818190422,37334,0,29424920,1
33764,376832,736,920,0,11349,8709180,0,9384,1104,6732669,4232,716872779831,39788584233,1474,0,151856,0
49422,92160,180,225,0,1906,907341,0,2880,270,859567,1035,1717970993,97221935579,230,0,1314000,0
52514,69632,136,170,0,1865,201943,0,2176,204,711398,782,106776211738,48215037930,274,0,1129840,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11786,4096,8,10,0,80,4242,0,98,12,79873,46,5007022737,0,10,0,7740,0
14110,4096,8,10,0,80,38391,0,102,12,94335,46,5007022737,0,10,0,18272,0
50385,4096,8,10,0,80,65077,0,90,12,81269,46,5007022737,0,10,0,54467,0
